In [ ]:
#!pip install -U torch torchvision torchtext torchdata pytest torchsummary portalocker

### Import libraries

In [ ]:
from data_chatbot import questions_answers, load_df, toTensor, show_lengths, tokenize_questions,tokenize_answers
from data_chatbot import pretrained_w2v # prepare_text
from models_chatbot import Seq2Seq
from vocab_chatbot import Vocab


from train_chatbot import pretrain, train
from apply_chatbot import answer



### Tests

In [ ]:
!python -m pytest -vv tests_chatbot.py

### Raw data

In [ ]:
# source_name = "poc"
source_name = "squad1"

In [ ]:
df_train, df_test = load_df(source_name=source_name)
df_train.head()

### Tokenized sentences

In [ ]:
questions_train_raw, questions_valid_raw, answers_train_raw, answers_valid_raw = questions_answers(source_name=source_name)
show_lengths(questions_train_raw, questions_valid_raw, answers_train_raw, answers_valid_raw)

### Filter data

In [ ]:
temp = [pair for pair in zip(questions_train_raw, answers_train_raw) if len(pair[1])>3]
questions_train_filt, answers_train_filt = map(list, zip(*temp))
temp = [pair for pair in zip(questions_valid_raw, answers_valid_raw) if len(pair[1])>3]
questions_valid_filt, answers_valid_filt = map(list, zip(*temp))
print(f"{len(questions_train_filt)} training questions and {len(questions_valid_filt)} valid questions remain.")

### Create vocabularies

In [ ]:
vQ = Vocab("Questions")
for sequence in [["<SOS>", "<EOS>"]] + questions_train_filt + questions_valid_filt:
    for token in sequence:
        vQ.indexWord(token)
vA = Vocab("Answers")
for sequence in [["<SOS>", "<EOS>"]] + answers_train_filt + answers_valid_filt:
    for token in sequence:
        vA.indexWord(token)
print(f"The source vocabulary contains {len(vQ.words)} and the target vocabulary contains {len(vA.words)} words.")

### Create vectors

In [ ]:
questions_train = tokenize_questions(questions_train_filt, vQ)
answers_train = tokenize_answers(answers_train_filt, vA)
questions_valid = tokenize_questions(questions_valid_filt, vQ)
answers_valid = tokenize_answers(answers_valid_filt, vA)

### Create model

In [ ]:
input_size = len(vQ.words)
hidden_size = 100
output_size = len(vA.words) 

dropout_E=0.0
dropout_D=0.0
teacher_forcing_ratio=0.0


model = Seq2Seq(input_size, hidden_size, output_size, vQ, vA, dropout_E, dropout_D, teacher_forcing_ratio)

### Utilize pretrained embeddings

In [ ]:
w2v = pretrained_w2v(init=False)
model = pretrain(model, vQ, vA, w2v)

#### => most_similar is not working after adding vector in gensim

### Train model

In [ ]:
epochs = 100
batch_size = 4
print_each = 1
lr = 0.01
weight_decay = 0
version = str(hidden_size)
train(epochs, batch_size, print_each, lr, weight_decay, model, version, questions_train, answers_train, questions_valid, answers_valid, vQ, vA)

In [13]:
model.eval()
for i in [1,2,200,-2,-1]:
    answer(model, questions_train[i].view(1,-1), vA, answers_train[i], vQ, 5)

question before tensor([[   0,    2,   62,   23, 5959,  169,  726,  132,    1]])
question: <SOS> what titl did darwin book ultim have <EOS>
answer: open of the bbc televis servic <EOS>
predicted_answer: manu amiibo nebraska bandurria witti highpow afar

question before tensor([[   0,   77,  388,  151, 2206,   15, 2179, 4747,    6,   48, 9776,  344,
          672, 1798, 1985,  433,  813, 2057,    7, 9777,    1]])
question: <SOS> how do charg accumul on each plate of a capacitor after an electr field ha develop across the dielectr <EOS>
answer: mae raekwon and eminem <EOS>
predicted_answer: repatri pineo csir tungurahua maximum

question before tensor([[   0,  139,  388,    7,   40, 6404, 6394, 1045,    4, 6405, 6372,    1]])
question: <SOS> when do the first facial hair present in pubesc male <EOS>
answer: limit to those test which are requir to verifi the nonfunct aspect of the system <EOS>
predicted_answer: nagasaki 737 smash smash smash smash smash smash anu amend van romanu infirm b

In [ ]:
answers_train

In [ ]:
import random
a=random.randint(0,3)
print(a)

In [ ]:
hallo